In [1]:
import sys
sys.path.insert(0, "/cndd/fangming/CEMBA/snmcseq_dev")

from multiprocessing import Pool,cpu_count
from functools import partial

from scipy import sparse
from scipy import stats
import importlib
from __init__ import *
from __init__jupyterlab import *
import snmcseq_utils
importlib.reload(snmcseq_utils)

import pickle
import datetime

In [2]:
today = datetime.date.today()
print(today)

2020-08-09


# Load data (pull from everywhere)

### old-to-new cell ids (mC) 

In [3]:
# old - new names
f = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze/snmc_and_snatac/metadata/MOp_Metadata.tsv.gz'
old2new = pd.read_csv(f, sep="\t")
old2new['old_name'] = old2new['AllcPath'].apply(lambda x: x.split('/')[-1][len("allc_"):-len(".tsv.gz")])
old2new['new_name'] = old2new['cell'].apply(lambda x: 'snmcseq_gene_{}'.format(x))

print(old2new.shape)
old2new.head()

(9941, 24)


cell                                           AllcPath  CCC_Rate  \
0     2C_M_0  /gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...   0.00511   
1     2C_M_1  /gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...   0.00531   
2    2C_M_10  /gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...   0.00581   
3   2C_M_100  /gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...   0.00634   
4  2C_M_1000  /gale/raidix/rdx-4/CEMBA_RS1/2C/CEMBA180409_2C...   0.00521   

   CG_Rate  CG_RateAdj  CH_Rate  CH_RateAdj  FinalReads  InputReads  \
0  0.77073     0.76955  0.02154     0.01651   1076259.0     2314862   
1  0.75683     0.75553  0.02196     0.01674   1197092.0     2590626   
2  0.75720     0.75578  0.02603     0.02034   1006630.0     2348484   
3  0.74984     0.74824  0.02718     0.02097   2026078.0     4599922   
4  0.77073     0.76953  0.02117     0.01604   2368464.0     5320502   

   MappedReads  ... MappingRate Pos96             Plate  Col96  Row96 Col384  \
0    1517338.0  ...    0.655477   A10  CEMBA180409_2C_1      9      0     18   
1    1709334.0  ...    0.659815   A10  CEMBA180409_2C_1      9      0     18   
2    1606410.0  ...    0.684020   A11  CEMBA180409_2C_2     10      0     20   
3    3154619.0  ...    0.685798    B1  CEMBA180409_2C_2      0      1      1   
4    3648786.0  ...    0.685797    F8  CEMBA180409_2C_3      7      5     15   

  Row384           Batch                                           old_name  \
0      0  CEMBA180409_2C  180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...   
1      1  CEMBA180409_2C  180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...   
2      0  CEMBA180409_2C  180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...   
3      2  CEMBA180409_2C  180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_1_CE...   
4     11  CEMBA180409_2C  180508_CEMBA_mm_P56_P63_2C_CEMBA180409_2C_3_CE...   

                 new_name  
0     snmcseq_gene_2C_M_0  
1     snmcseq_gene_2C_M_1  
2    snmcseq_gene_2C_M_10  
3   snmcseq_gene_2C_M_100  
4  snmcseq_gene_2C_M_1000  

[5 rows x 24 columns]

### enhancer-cell matrix (mC)

In [4]:
ti = time.time()

# cells 
f = '/cndd2/fangming/projects/scf_enhancers/enhancer_cell_level/mcg/combined_enh_cell_mc_mats.cell'
enhancer_cell_mc_mats_cell = snmcseq_utils.import_single_textcol(f)
print(len(enhancer_cell_mc_mats_cell))
enhancer_cell_mc_mats_cell = old2new.set_index('old_name').reindex(enhancer_cell_mc_mats_cell)['new_name'].values
print(len(enhancer_cell_mc_mats_cell))
print(enhancer_cell_mc_mats_cell[:5])

# regions
f = '/cndd2/fangming/projects/scf_enhancers/enhancer_cell_level/mcg/combined_enh_cell_mc_mats.region'
enhancer_cell_mc_mats_region = pd.read_csv(f, sep='\t') 
print(len(enhancer_cell_mc_mats_region))
print(enhancer_cell_mc_mats_region.head())

# takes ~3min to load
# enhancer-cell-matrix
f = '/cndd2/fangming/projects/scf_enhancers/enhancer_cell_level/mcg/combined_enh_cell_mc_mats.pkl'
with open(f, 'rb') as handle:
    enhancer_cell_mc_mats = pickle.load(handle)
    
for col in enhancer_cell_mc_mats.keys():
    # was originally cell-by-enhancer; turn into enhancer-by-cell
    enhancer_cell_mc_mats[col] = enhancer_cell_mc_mats[col].T 
    print(col, enhancer_cell_mc_mats[col].shape)
    
print(time.time()-ti)

9931
9931
['snmcseq_gene_3C_M_0' 'snmcseq_gene_3C_M_1' 'snmcseq_gene_3C_M_2'
 'snmcseq_gene_3C_M_3' 'snmcseq_gene_3C_M_4']
233514
  chr    start      end
0   1  3083769  3083816
1   1  3112234  3112467
2   1  3112770  3112986
3   1  3113717  3114035
4   1  3118790  3118935
mCG (233514, 9931)
CG (233514, 9931)
mCH (233514, 9931)
CH (233514, 9931)
19.020012140274048


### gene-cell matrix (mCH)

In [5]:
# gene-cell matrix (rna)

ti = time.time()
f_mc = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_mCH_raw.npz'
f_c = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_CH_raw.npz'
fgene = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_raw.gene'
fcell = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/snmcseq_gene_raw.cell'

gene_cell_mch_mat = snmcseq_utils.load_gc_matrix_methylation(fgene, fcell, f_mc, f_c)

print(gene_cell_mch_mat.data['mc'].shape, gene_cell_mch_mat.data['c'].shape)
print(time.time()-ti)

(55487, 9366) (55487, 9366)
37.483447551727295


### enhancer-gene linkage

In [6]:
# enhancer-gene linkage
ti = time.time()
f = '/cndd2/fangming/projects/scf_enhancers/results/200521_to_evals.tsv'
enhancer_gene_to_eval = pd.read_csv(f, sep='\t')
# f = '/cndd2/fangming/projects/scf_enhancers/results/200521_to_evals_shuffled.tsv'
# enhancer_gene_to_eval_shuffled = pd.read_csv(f, sep='\t')

print(enhancer_gene_to_eval.shape)
print(time.time()-ti)
enhancer_gene_to_eval.head()

(2589994, 18)
7.6826252937316895


gene     ens       val      val2  val_clustershuffled  \
0  ENSMUSG00000000001  147520 -0.157895  0.127913            -0.169712   
1  ENSMUSG00000000001  147522  0.027465 -0.307145             0.064887   
2  ENSMUSG00000000001  147523 -0.040376  0.011489            -0.155487   
3  ENSMUSG00000000001  147527 -0.070795  0.111719            -0.088084   
4  ENSMUSG00000000001  147528  0.012583  0.150673            -0.036218   

   val2_clustershuffled    dist   pvalues  fdr<100kb  fdr<200kb  fdr<500kb  \
0             -0.049349  995118  0.288079   0.674683   0.736811   0.827139   
1              0.105591  985740  0.561881   0.830849   0.867082   0.917381   
2              0.014772  983725  0.457686   0.780196   0.825259   0.889141   
3              0.184594  970444  0.411564   0.755088   0.804341   0.874375   
4              0.147609  969372  0.539123   0.819981   0.858026   0.911467   

   fdr<100kb shuffled (cluster label)  pvalues2  fdr2<100kb  fdr2<200kb  \
0                            1.647931  0.383141    0.745977    0.797076   
1                            0.958300  0.935264    0.973809    0.979844   
2                            1.089312  0.580703    0.841673    0.874844   
3                            1.183332  0.410358    0.761056    0.809464   
4                            0.979843  0.345993    0.725803    0.780447   

   fdr2<500kb  fdr2<100kb shuffled (cluster label)  val_both  
0    0.872822                             1.955898 -0.285808  
1    0.987476                             0.959470  0.334610  
2    0.922832                             1.273339 -0.051866  
3    0.880982                             1.812516 -0.182514  
4    0.862033                             2.191506 -0.138090

# Organize data (and save to local folder) 

### match cells 

In [7]:
common_cells = np.intersect1d(enhancer_cell_mc_mats_cell, 
                              gene_cell_mch_mat.cell
                             ) 
print(len(common_cells))


9364


### match gene and enhancers 

In [8]:
# just need first number greater than the second
print(len(gene_cell_mch_mat.gene), len(enhancer_gene_to_eval['gene'].unique()))
print(len(enhancer_cell_mc_mats_region), len(enhancer_gene_to_eval['ens'].unique()))

55487 18988
233514 151577


### match dimensions of matrices 

In [9]:
cell_idx_mch_gene = snmcseq_utils.get_index_from_array(gene_cell_mch_mat.cell, common_cells)
X_mch = gene_cell_mch_mat.data['mc'].tocsc()[:,cell_idx_mch_gene]
X_ch = gene_cell_mch_mat.data['c'].tocsc()[:,cell_idx_mch_gene]
print(X_mch.shape, X_ch.shape)

(55487, 9364) (55487, 9364)


In [10]:
cell_idx_mc = snmcseq_utils.get_index_from_array(enhancer_cell_mc_mats_cell, common_cells)
Y_mcg = enhancer_cell_mc_mats['mCG'].tocsc()[:,cell_idx_mc]
Y_cg = enhancer_cell_mc_mats['CG'].tocsc()[:,cell_idx_mc]
print(Y_mcg.shape, Y_cg.shape)

(233514, 9364) (233514, 9364)


In [11]:
# cell_idx_xaxis = snmcseq_utils.get_index_from_array(cell_cell_knn_xaxis, common_rna_cells)
# cell_idx_yaxis = snmcseq_utils.get_index_from_array(cell_cell_knn_yaxis, common_mc_cells)

# knn_xy = cell_cell_knn_mat.tocsr()[cell_idx_xaxis,:].tocsc()[:,cell_idx_yaxis] # x-by-y
# print(knn_xy.shape)

In [12]:
# cell_idx_axis = snmcseq_utils.get_index_from_array(cell_cell_knn_mat_within_axis, common_rna_cells)
# knn_xx = cell_cell_knn_mat_within.tocsr()[cell_idx_axis,:].tocsc()[:,cell_idx_axis] # x-by-x
# print(knn_xx.shape)

In [13]:
# print(knn_xx.sum(axis=1)) # weighted ka=5, p=0.7

# Save these raw matrices we need later 
- will be used over and over later 

In [14]:
# 4 one-dimensional objects
# cell_x
common_cells

# gene
gene_cell_mch_mat.gene
# enh
enhancer_cell_mc_mats_region

# 5 matrices [all in sparse format]
# cell (mc) by gene RNA 
X_mch
X_ch

# cell (mC) by gene mC
Y_mcg
Y_cg

# # knn_xy (cell_in_x) by (cell_in_y); matched with above 
# knn_xy

# # knn_xx (cell_in_x)
# knn_xx

<233514x9364 sparse matrix of type '<class 'numpy.int64'>'
	with 382503141 stored elements in Compressed Sparse Column format>

In [15]:
bundle = (
    common_cells,  
    gene_cell_mch_mat.gene, enhancer_cell_mc_mats_region,
    X_mch, X_ch, Y_mcg, Y_cg, 
#     knn_xy, knn_xx,
    )
bundle_fnames = (
    'cell_snmcseq_gene.txt',
    'gene_snmcseq_gene.txt',
    'enh_snmcseq_gene.tsv',
    
    'mat_genebody_mch_snmcseq_gene.npz',
    'mat_genebody_ch_snmcseq_gene.npz',
    'mat_enhancer_mcg_snmcseq_gene.npz',
    'mat_enhancer_cg_snmcseq_gene.npz',
#     'knn_smarter_cells_snmcseq_gene.npz',
#     'knn_smarter_cells.npz',
)

for item, fname in zip(bundle, bundle_fnames):
    print(type(item), item.shape, fname)

<class 'numpy.ndarray'> (9364,) cell_snmcseq_gene.txt
<class 'numpy.ndarray'> (55487,) gene_snmcseq_gene.txt
<class 'pandas.core.frame.DataFrame'> (233514, 3) enh_snmcseq_gene.tsv
<class 'scipy.sparse.csc.csc_matrix'> (55487, 9364) mat_genebody_mch_snmcseq_gene.npz
<class 'scipy.sparse.csc.csc_matrix'> (55487, 9364) mat_genebody_ch_snmcseq_gene.npz
<class 'scipy.sparse.csc.csc_matrix'> (233514, 9364) mat_enhancer_mcg_snmcseq_gene.npz
<class 'scipy.sparse.csc.csc_matrix'> (233514, 9364) mat_enhancer_cg_snmcseq_gene.npz


In [16]:
f = '/cndd2/fangming/projects/scf_enhancers/data/organized_cell_level/version_mc_only_aug9'
ti = time.time()
with snmcseq_utils.cd(f):
    for item, fname in zip(bundle, bundle_fnames):
        print(type(item), item.shape, fname, time.time()-ti)
        #  save all as pickle file
        with open(fname, "wb") as fh:
            pickle.dump(item, fh)

<class 'numpy.ndarray'> (9364,) cell_snmcseq_gene.txt 0.023510456085205078
<class 'numpy.ndarray'> (55487,) gene_snmcseq_gene.txt 0.046225786209106445
<class 'pandas.core.frame.DataFrame'> (233514, 3) enh_snmcseq_gene.tsv 0.08065390586853027
<class 'scipy.sparse.csc.csc_matrix'> (55487, 9364) mat_genebody_mch_snmcseq_gene.npz 0.11079955101013184
<class 'scipy.sparse.csc.csc_matrix'> (55487, 9364) mat_genebody_ch_snmcseq_gene.npz 3.7308096885681152
<class 'scipy.sparse.csc.csc_matrix'> (233514, 9364) mat_enhancer_mcg_snmcseq_gene.npz 8.161460161209106
<class 'scipy.sparse.csc.csc_matrix'> (233514, 9364) mat_enhancer_cg_snmcseq_gene.npz 15.190521955490112
